# Graph Coloring Analysis Project

This project looks at implementing an algorithm in multiple ways to solve a problem, analyzing the
algorithms’ implementations for running time, and testing and your implementations to show they
match your analysis. The project also looks at testing to determine how well the multiple
implementations solve the problem.<br/>
The particular problem for the project is scheduling via graph coloring. For the first part of the project,
different conflict graphs which can represent real world problems will be created and saved in files. For
the second part of the problem, various coloring orders will be implemented and analyzed for runtime
efficiency and coloring efficiency. These efficiencies are intimately related to the data structures used
for both the input graph data and the intermediate data necessary for the ordering and coloring
algorithms.

## Part 1

### Purpose

Part 1 of the project is used to create different conflict graphs. Conflict graphs represent realworld problems. For example, one may be thought of as a set of vertices representing courses that students may take. An edge is present between two vertices (courses) in the graph if a student is taking both courses. This edge indicates they may not be scheduled at the same time.<br/>
Once the graph is colored, the color of each vertex represents its timeslot and the total number of colors would represent the total number of required timeslots for that coloring.<br/>
The vertices which have a conflict in this project will be determined based on graph types and a random number generator with different distributions.

-------------------------------------------
#### My Notes
Vertex is a class<br/>
Edge means a student is taking both classes (the vertices connected) -> that can't happen, represents a conflict, need =/ colors<br/>
graphs are generated by graph types and random generator w/ =/ distributions<br/>

-------------------------------------------

In [1]:
# Imports
import numpy as np
from scipy.stats import skew
import random
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
    '''
    
    
    Parameters
    ----------
    id: int
        The value of the vertex    
    '''
    

'\n\n\nParameters\n----------\nid: int\n    The value of the vertex    \n'

### Helper Functions

In [3]:
def write_file(adj):
    '''
    Writes the adjacency list to a file in the format:
        N - Number of vertices in the graph with V vertices and E edges. 
        P[] = Pointer for each vertex V, 1 <= V <= N denoting the starting point in E[] of the list of 
            vertices adjacent to vertex V. That is, the vertices adjacent to vertex V are indicated in 
            locations E[P[V]], E[P[V]+1], ..., E[P[V+1]-1].  
        E[] = list of distinct graph edges (length = 2E) 
    
    Parameters
    ----------
    adj: list of lists
        Adjacency list representing the graph    
    '''
    
    file = open("graph.txt", "w")
    
    # Print Number of Vertices
    file.write(str(len(adj)) + '\n')

    # Print Starting Locations of Vertices' Edges
    prevLoc = len(adj) + 1
    file.write(str(prevLoc) + '\n')

    for v in range(len(adj)):
        if v != 0:
            prevLoc = prevLoc + np.count_nonzero(adj[v-1]) - 1
            file.write(str(prevLoc) + '\n')


    # Print All Edges
    for v in range(len(adj)):
        for e in range(len(adj[v])):
            if e != 0:
                file.write(str(adj[v][e]) + '\n')
    
    file.close()

In [4]:
def print_graph(adj):
    '''
    Prints the graph in a picture using the library networkx
    
    Parameters
    ----------
    adj: list of lists
        Adjacency list representing the graph
    '''
    G = nx.Graph()
    
    # Get total number of vertices
    v_num = len(adj)
    
    # Add Vertices to graph
    G.add_nodes_from(range(1, v_num+1))
    
    # Add edges (from adjacency list) to graph
    for v in range(len(adj)):
        for e in range(len(adj[v])):
            if e != 0:
                G.add_edge(adj[v][0], adj[v][e])
                # G.add_edge(adj[v][0], adj[v][e], color='r')
    
    # Print the graph
    nx.draw(G, with_labels=True, font_weight='bold')

### Graph Generation

In [5]:
def complete(V):
  # Create Adjacency List
  adj = [[0 for i in range(V)] for j in range(V)]

  # Fill in vertices at the first column in all rows
  for index in range(V):
    adj[index][0] = index + 1
  
  index = 0
  found = False

  for r in range(len(adj)):
    found = False
    for c in range(len(adj[r])):
        # Skip over first column
        if c == 0:
          index = adj[r][c]
          # print(index) # CORRECT
        
        # Iterate V-1 times, fill other spots with all values except index
        else:          
          # Check if the current edge is the same as the vertex
          if c == index:
            found = True

          # If it has found the vertex, then just skip it and all next values
          # will be + 1
          if found:
            adj[r][c] = c + 1
          else:
            adj[r][c] = c

  return adj

In [6]:
def cycle(V):
    adj = [[0 for i in range(3)] for j in range(V)]

    for index in range(V):
        adj[index][0] = index + 1
    
    for r in range(len(adj)):
        # Each vertex only has 2 edges in a cycle
        # These edges are either + or - 1 in relation to the value of the vertex
        adj[r][1] = adj[r][0] - 1
        adj[r][2] = adj[r][0] + 1

        # Check if the value is not 0 or over the number of vertices
        if adj[r][1] == 0:
            adj[r][1] = V
        
        if adj[r][2] == V+1:
            adj[r][2] = 1
    
    return adj

In [7]:
def uniform_dist(V, E, show_graph=False):
    
    # Pick random vertices
    # Get a larger number of vertices to account for repeated edges or self loops
    edges = np.random.randint(1, V+1, E*8)
    
    adj = [[j+1] for j in range(V)]
    
    # print('Length of Edges: ', len(edges))
    # print('Chosen Edges: ', edges)
    
    counter = 0
    for i in range(len(edges)):
        if counter == E:
            break
        if i % 2 == 0:
            if edges[i] != edges[i+1]:
                if adj[edges[i]-1][0] != edges[i+1]:
                    if edges[i+1] not in adj[edges[i]-1]:
                        adj[edges[i]-1].append(edges[i+1])
                        adj[edges[i+1]-1].append(edges[i])
                        counter+=1
        
    
    if show_graph:
        plt.hist(edges, bins=50)
        plt.show()
    
    return adj

In [8]:
def skewed_dist(V, E, show_graph=False):
    # Create Adjacency List
    # adj = [[i+1] for i in range(V)]
    
    
    edges = np.random.triangular(0, 0, V, E*8)
    
    edges = [int(x) for x in edges]
    
    adj = [[j+1] for j in range(V)]
    
    # print('Length of Edges: ', len(edges))
    # print('Chosen Edges: ', edges)
    
    counter = 0
    for i in range(len(edges)):
        if counter == E:
            break
        if i % 2 == 0:
            if edges[i] != edges[i+1]:
                if adj[edges[i]-1][0] != edges[i+1]:
                    if edges[i+1] not in adj[edges[i]-1]:
                        adj[edges[i]-1].append(edges[i+1])
                        adj[edges[i+1]-1].append(edges[i])
                        counter+=1
        
    
    # print(adj)
    if show_graph:
        plt.hist(edges, bins=50)
        plt.show()
    
    return adj

In [9]:
def binomial_dist(V, E, show_graph=False):
    # Create Adjacency List
    
    edges = [np.random.binomial(V, 0.5) for i in range(E*8)]
    
    edges = [int(x) for x in edges]
    
    adj = [[j+1] for j in range(V)]
    
    # print('Length of Edges: ', len(edges))
    # print('Chosen Edges: ', edges)
    
    counter = 0
    for i in range(len(edges)):
        if counter == E:
            break
        if i % 2 == 0:
            if edges[i] != edges[i+1]:
                if adj[edges[i]-1][0] != edges[i+1]:
                    if edges[i+1] not in adj[edges[i]-1]:
                        adj[edges[i]-1].append(edges[i+1])
                        adj[edges[i+1]-1].append(edges[i])
                        counter+=1
    
    
    if show_graph:
        plt.hist(edges, bins=50)
        plt.show()
    
    return adj

### Testing Max Cases

In [52]:
%time display(graph = uniform_dist(10000, 2000000)) # ~30 seconds

CPU times: total: 39.9 s
Wall time: 40.5 s


In [53]:
%time display(graph = skewed_dist(10000, 2000000)) # ~30 seconds

KeyboardInterrupt: 

In [ ]:
%time display(graph = binomial_dist(10000, 2000000)) # ~40 seconds

CPU times: total: 55 s
Wall time: 55.4 s


In [ ]:
%time display(graph = complete(10000)) # ~10 seconds

CPU times: total: 13.1 s
Wall time: 13.2 s


In [ ]:
%time display(graph = cycle(10000)) # Instantaneous

CPU times: total: 15.6 ms
Wall time: 8 ms


### Visualization Functions

In [10]:
def printGraph(adj):
  print(adj)
  print()
  
  # Print Number of Vertices
  print(len(adj))

  # Print Starting Locations of Vertices' Edges
  prevLoc = len(adj) + 1
  print(prevLoc)

  for v in range(len(adj)):
    if v != 0:
      prevLoc = prevLoc + np.count_nonzero(adj[v-1]) - 1
      print(prevLoc)


  # Print All Edges
  for v in range(len(adj)):
    for e in range(len(adj[v])):
      if e != 0:
        print(adj[v][e])

### Input

* V = Number of vertices. (MAX = 10,000)
* E = Number of conflicts between pairs of vertices for random graphs. (MAX = 2,000,000)
* G = COMPLETE | CYCLE | RANDOM (with DIST below)
* DIST = UNIFORM | SKEWED | YOURS

### Output

* E[] = adjacency list of distinct course conflicts (length = 2M)
* P[] = Pointer for each course I, 1 <= I <= N denoting the starting point in E[] of the list of
courses in conflict with course I. 
  * That is, the conflicts for course I are indicated in locations E[P[I]], E[P[I]+1], …, E[P[I+1]-1].

### Procedure

The vertices for conflicts shall be chosen using a pseudo random number generator according to one of the specified distributions. These distributions are uniform, skewed, and one other of your choosing.<br/>
The uniform distribution assumes each vertex is equally likely to be chosen for a conflict. The skewed distribution assumes lower numbered vertices are linearly more likely than higher numbered vertices.<br/>
Graphs of these distributions are shown below. <br/><br/>
![Graph1](Probability-of-Choosing-Graphs.png)<br/>
**You may use a built-in uniform distribution pseudo-random number generator, but must base the
other distributions on the output of this built-in generator.**

In [11]:
def create_graph(V, E, G, DIST=''):    
    if G == 'COMPLETE':
        graph = complete(V)
    elif G == 'CYCLE':
        graph = cycle(V)
    else: # G == 'RANDOM'
        if DIST == 'UNIFORM':
            graph = uniform_dist(V, E)
        elif DIST == 'SKEWED':
            graph = skewed_dist(V, E)
        else: # DIST == 'FREQUENCY'
            graph = binomial_dist(V, E)
    
    write_file(graph)

## Part 2

### Input: A file like the output in Part 1

* P[] = Pointer for each course I, 1 <= I <= N denoting the starting point in E[] of the list of
courses in conflict with course I.
  * That is, the conflicts for course I are indicated in locations E[P[I]], E[P[I]+1], …, E[P[I+1]-1].
* E[] = adjacency list of distinct course conflicts (length = 2M).

### Output

* For each vertex the color, original degree, (and degree when deleted for the smallest last ordering). These should be printed in the order colored.
* Total number of colors used, the average original degree, and the maximum “degree when deleted” value for the smallest last ordering, and the size of the terminal clique for the smallest last ordering.
* Any other summary data you wish to include.
* An output file of the format where each line is VERTEX_NUMBER, COLOR_NUMBER.

### Debugging Functions

In [12]:
def print_vertices(adj):
    s = '['
    for i in range(len(adj)):
        s += '['
        for j in range(len(adj[i])):
            s += str(adj[i][j].id)
            if j != len(adj[i])-1:
                s += ', '
        s += ']'
        if i != len(adj)-1:
            s += ', '
    s+= ']'
    
    print(s)

def print_degrees(adj):
    s = '['
    for i in range(len(adj)):
        s += '['
        for j in range(len(adj[i])):
            s += str(adj[i][j].id) + ': ' + str(adj[i][j].degree)
            if j != len(adj[i])-1:
                s += ', '
        s += ']'
        if i != len(adj)-1:
            s += ', '
    s+= ']'
    
    print(s)

### Helper Functions

In [13]:
class Vertex:
    '''
    Vertex class. Instead of storing just an int in the adjacency list,
    we store a Vertex, which holds much more information.

    Parameters
    ----------
    id: int
        The value of the vertex
    adj_idx: int
        The index of the vertex in the adjacency list
    degree: int
        The degree of the vertex. Calculated by the length of the list at the index of the vertex in the adjacency list
    idx_degree: int
        Optional parameter. The index of the vertex in the degree list.
    '''
    
    def __init__(self, id, adj_idx, degree, idx_degree=None, deleted=False, color=None):
        self.id = id
        self.adj_idx = adj_idx
        self.degree = degree
        self.idx_degree = idx_degree
        self.deleted = deleted
        self.color = color

In [14]:
def read_file(debug=False):
    '''
    Reads the file (created in Part 1) adnd returns the adjacency list filled with Vertex objects.

    Parameters
    ----------
    debug: bool
        Optional parameter. If true, prints the helpful statements.
    '''
    
    file = open('graph.txt', 'r')
    num_lines = sum(1 for line in file)
    file.close()
    
    file = open("graph.txt", "r")
    
    # Adjacency list to be filled with Vertices
    adj = []
    edge_idxs = []
    
    # Get number of vertices
    num_vertices = int(file.readline())
    if debug: print('Number of Vertices: ', num_vertices)
    
    # Fill adjacency list with the vertices
    s = 'Edge Indexes: '
    for i in range(num_vertices):
        edge_idxs.append(int(file.readline()))
        if debug: s += (str(edge_idxs[i]) + ', ')
    edge_idxs.append(num_lines)
    if debug: s += str(edge_idxs[i+1])
    if debug: print(s)
    
    curr_line = num_vertices + 1
    nxt_line = edge_idxs[1]
    curr_edge_idx = 1
    if debug: print('Line after getting indexes: ', curr_line)
    if debug: print('Line for next index: ', nxt_line)
    
    # Fill adjacency list with the vertices
    for i in range(num_vertices):
        adj.append([Vertex(i+1, i, (edge_idxs[i+1]-edge_idxs[i]))])
        
    if debug: print('Just Vertices: ')
    if debug: print_vertices(adj)
    if debug: print('Degree Vertex 1: ', adj[0][0].degree)
    if debug: print('Degree Vertex 2: ', adj[1][0].degree)
    if debug: print('Degree Vertex 3: ', adj[2][0].degree)
    if debug: print('Degree Vertex 4: ', adj[3][0].degree)
    if debug: print('Degree Vertex 5: ', adj[4][0].degree)
    
    # Fill adjacency list with the edges
    curr_v = 0
    for i in range(num_vertices):
        for j in range(adj[i][0].degree):
            adj[i].append(adj[int(file.readline())-1][0])
        curr_v += 1
            
    if debug: print('Finished List: ')
    if debug: print_vertices(adj)
    
    file.close()
    return adj

In [31]:
def print_edges(adj):
    s = '['
    for i in range(len(adj)):
        s += '['
        for j in range(len(adj[i])):
            if type(adj[i][j]) == int:
                s += str(adj[i][j]) + ' Degrees: '
            else:
                s += str(adj[i][j].id)
            if j != len(adj[i])-1:
                s += ', '
        s += ']'
        if i != len(adj)-1:
            s += ', '
    s+= ']'
    print(s)

def get_degree_list(adj, debug=False):
    high_deg = 0
    for i in adj:
        if high_deg <= i[0].degree:
            high_deg = i[0].degree
    if debug: print('High Degree: ', high_deg)
    
    edge_l = [[i] for i in range(high_deg+1)]
    
    for i in adj:
        edge_l[i[0].degree].append(i[0])
    
    print_edges(edge_l)
    
    return edge_l

test3 = read_file()
get_degree_list(test3, True)
print()

High Degree:  3
[[0 Degrees: ], [1 Degrees: ], [2 Degrees: , 2, 3, 5], [3 Degrees: , 1, 4]]



In [16]:
def fill_vertices(adj):
    '''
    Creates an adjacency list of Vertex objects
    
    Parameters
    ----------
    adj: list of lists
        Adjacency list representing the graph
    '''
    
    l = [[] for i in range(len(adj))]
    
    for i in range(len(adj)):
        l[i].append(Vertex(adj[i][0], (i, 0), len(adj[i])-1))
        for j in range(len(adj[i])):
            if j != 0:
                degree = len(adj[adj[i][j]-1]) - 1
                l[i].append(Vertex(adj[i][j], (i, j), degree))
                
    return l

### Procedure

CS-7350 students are to use six different methods for ordering the vertices in the graph. One method all
students are to use is the smallest last ordering given below, another is the smallest original degree last
and the final one for all students is a uniform random ordering. The other orderings are of your own
choosing. Then you are to assign the minimum color to each vertex in the order determined by each
ordering so that it doesn’t conflict with the other vertices already colored. You will then report on the
following criteria for the different ordering methodologies:
* Asymptotic running time for Smallest Last Ordering and Coloring
* Total number of colors needed for each ordering for various graph types
* Report any other interesting metric for each ordering

### Smallest Vertex Ordering

The following format for the values of variables in various fields of the data node for each vertex may be used to save storage space. You may also split fields into different ones to avoid overloading a field for code readability and maintenance.<br/><br/>
![Table Example](Table-Example.png)<br/>
1. Establish the pointers to the lists of vertices of degree j, 0 <= j <= N-1, and any other pointers
or variables needed.
2. Create the code to iteratively delete a vertex of smallest degree, updating the third and
fourth fields of each data node to relate to the remaining graph, adding the vertex deleted
to the ordered list of vertices deleted.
3. After all vertices have been deleted, scan and assign colors (session periods) to each vertex
in an order opposite to the order deleted, assigning for a “color” the smallest non-negative
integer not previously assigned to any adjacent vertex. The output associated with each
vertex may be printed as the coloring is assigned.
4. Print any further output and summary information for the schedule.

In [17]:
# Watch the video where he explains the algorithm and the lists and when to delete stuff

for vertices in graph
    delete the vertex with the smallest degree
    Store that vertex and its degree on a list
    Update vertices that were connected to that vertex by decreasing their degree
    
    Stop when you remove the last vertex which should have a degree of 0
    
    When all vertices have the same number of edges, you have a complete subgraph
        Record those vertices -> terminal clique
        
Looking at the Vertex with the highest degree on the list means you might need degree+1 colors
The number of vertices in the terminal clique is the same number as the best you can color the graph

Color them now

for vertices in list.reverse
    for vertices connected to current vertex
        remove their colors from available colors
    
    color curr with the first color in available colors list
    

SyntaxError: expected ':' (4161757362.py, line 3)

In [ ]:
def smallest_vertex(adj, ):
    

In [ ]:
t = Vertex(5, 0, 3)

def get_least_degree(adj):
    smallest_idx = np.inf
    smallest = np.inf
    
    for v in range(len(adj)):
        if adj[v][0].degree != 0:
            if adj[v][0].degree < smallest:
                smallest = adj[v][0].degree
                smallest_idx = (v, 0)
            
    return smallest_idx

In [ ]:
def print_colors(adj):
    G = nx.Graph()
    
    # Get total number of vertices
    v_num = len(adj)
    
    # Add Vertices to graph
    G.add_nodes_from(range(1, v_num+1))
    
    # Add edges (from adjacency list) to graph
    for v in range(len(adj)):
        for e in range(len(adj[v])):
            if e != 0:
                G.add_edge(adj[v][0].id, adj[v][e].id)
                # G.add_edge(adj[v][0], adj[v][e], color='r')
    
    # Print the graph
    nx.draw(G, with_labels=True, font_weight='bold')

For additional output with METHOD 1 you should include
* A graph indicating the degree when deleted on the vertical axes and the order colored on the x-axis.
* The maximum degree when deleted.
* The size of the terminal clique.
* Any other summary data you wish to include.

| Vertex 1    | Field 1     | Field 2     | Field 3     | Field 4     |
| ----------- | ----------- | ----------- | ----------- | ----------- |
| Header      | Title       | Syntax      | Description | ds          |
| Paragraph   | Text        | Syntax      | Description | sdaa        |

### Smallest Original Degree Last

The Smallest Original Degree Last method is a subset of the smallest last ordering. You should
determine the vertices to color based on their original degree, but not remove them from the
graph as you do it. This should run in THETA(V+E).

### Testing

You should test your program in such a fashion to convince me it operates as expected. Test input files
will also be provided.